In [ ]:
#'words' is a list containing regions of image which could be words

import cv2
import numpy as np
import os
import Word

def findWords(binary, path):
	kernelr2 = np.ones((3,9), np.uint8)

	#Pre-process input image
	eroded = cv2.erode(binary, kernelr2, iterations = 1)
	inverted = cv2.bitwise_not(eroded)

	#Detect words
	rectangles = []
	contours,hierarchy = cv2.findContours(inverted, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
	for contour in contours:
		x,y,w,h = cv2.boundingRect(contour)
		row, col = eroded.shape
		if(w*h<row*col/220):#300
			continue
		rectangles.append([x,y,w,h])

	#Scan and store words from left to right
	words = []
	rectangles.sort(key = lambda x: x[0])
	for rectangle in rectangles:
		x,y,w,h = rectangle
		word = binary[y:y+h,x:x+w].copy()
		words.append(word)

	#Make folders to save images and search for letters
	i=1
	for word in words:
		word_path = path + "/Word%d" %i
		os.mkdir(word_path)
		cv2.imwrite(word_path + "/word%d.jpg" %i, word)
		i+=1
		Word.findLetters(word, word_path)